In [53]:
import pandas as pd 
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error as rmse

## Q1. Downloading the data
We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".
Download the data for January and February 2023.
Read the data for January. 

How many columns are there?

In [54]:
df = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")
df.shape[1]

19

## Q2. Computing duration
Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

In [55]:
df['duration'] = ((df.tpep_dropoff_datetime - df.tpep_pickup_datetime).dt.total_seconds()/60)

df.duration.std()

np.float64(42.59435124195458)

## Q3. Dropping outliers
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [56]:
df_excl = df.loc[df.duration.between(1,60,inclusive='both')]

df_excl.shape[0] / df.shape[0]

0.9812202822125979

## Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

* Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
* Fit a dictionary vectorizer
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

In [57]:
categorical = ['PULocationID', 'DOLocationID']
target = 'duration'

X = df_excl[categorical].astype(str).to_dict(orient='records')
y = df_excl[target].values

dv = DictVectorizer()
X_train = dv.fit_transform(X)
X_train.shape[1]

515

## Q5. Training a model
Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters, where duration is the response variable
Calculate the RMSE of the model on the training data

What's the RMSE on train?

In [58]:
lr = LinearRegression()
lr.fit(X_train,y)
y_train_pred = lr.predict(X_train)
rmse(y, y_train_pred)

7.649261934850555

## Q6. Evaluating the model
Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [59]:
PATH = " https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet"

df_val = pd.read_parquet(PATH)
df_val['duration'] = ((df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime).dt.total_seconds()/60)
df_val_excl = df_val.loc[df_val['duration'].between(1,60,inclusive='both')]
X_val_dict = df_val_excl[categorical].astype(str).to_dict(orient='records')
y_val = df_val_excl[target].values
X_val = dv.transform(X_val_dict)
y_val_pred = lr.predict(X_val)
rmse(y_val, y_val_pred)

7.811817745843695